In [16]:
:opt no-lint

In [1]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

# 18 Monad
## 18.1 Monad
Used by current Haskell implementation for IO actions. They are applicative functors with particularities.
## 18.2 Sorry—a monad is not a burrito
In newer GHC versions, the type class definition has an `Applicative` constraints.
Hence, `fmap` can be written using monadic operations:

In [3]:
fmap f xs = xs >>= return . f

### Core operations
The Monad type class defines three core operations:
* bind: `>>=`
* next: `>>`
* `return` (same as `pure`)
You only need to define `>>=` for a minimally complete `Monad` instance.

### The novel part of Monad
`>>=` is similar to `<$>` and `<*>`. 
The difference is that `Monad` can alter the structure, while `Functor` and `Applicative` leave it untouched.
 `Monad`, in a sense, is a generalization of `concat`.
Hence, we can define a `Monad` by implementing `join`.
#### The answer is the exercise
Write bind in terms of `fmap` and `join`.

In [4]:
import Control.Monad (join)

bind :: Monad m => (a -> m b) -> m a -> m b
bind f ma = join $ fmap f ma

### What Monad is not
`Monad` is an abstract concept. Try to understand it from the perspective of the `IO Monad` may lead to limited intuitions.

A monad is not:
* Impure: monadic functions are pure functions. `IO` is an abstract datatype that allows for impure actions and it has a `Monad` instance.
* An embedded language for imperative programming. While monads are often used for sequencing actions, there are commutative monads that do not order actions.
* A value: the type class describes a specific relationship between elements in a domain and defines some operations over them.
4. About strictness: the monadic operations of bind and return are non-strict.
Using monads also doesn’t require knowing math. Or category theory. It does not require mystical trips to the tops of mountains or starving oneself in a desert somewhere.

### Monad also lifts!
For backward compatibility reasons, the `Monad` class also includes a set of `lift` functions that are the same as the ones we already saw in `Applicative`.
## 18.3 do syntax and monads
`*>` is a sequencing operation equivalent to `>>` but defined with an `Applicative` constraints.

In [5]:
import Control.Applicative ((*>))

sequencing :: IO ()
sequencing = do
  putStrLn "blah"
  putStrLn "another thing"

is desugared into:

In [6]:
sequencing'' :: IO ()
sequencing'' =
  putStrLn "blah" *>
  putStrLn "another thing"

and

In [7]:
binding :: IO ()
binding = do
  name <- getLine
  putStrLn name

is desugared into:

In [8]:
binding' :: IO ()
binding' =
  getLine >>= putStrLn

#### When fmap alone isn’t enough
In order to merge `IO` effects is necessary to use a monadic join instead of a `fmap` (it is not able to operate on the structure in order to flatten it). Also for `IO`, monadic actions are still pure, and the sequencing operations are ways of
nesting lambdas. `IO` allows for side effects, but since those effects are constrained within the `IO` type, all the rest of it is still a pure lambda calculus.
## 18.4 Examples of Monad use
### List
The `Monad` instance of `[]` essentially apply a function and concatenate the results into a single list.
### Maybe
A sequence of `Maybe` `Monad` has a fail-fast semantic. The differences with `Applicative` instance are:
1. With the `Applicative`, each Maybe computation fails or succeeds independently of one another. You’re lifting functions that are also `Just` or `Nothing` over `Maybe` values.
2. With the `Monad`, computations contributing to the final result can choose to return `Nothing` based on previous computations.

### Either
The `Either` `Monad` short-circuits on the first `Left` value. It must because in the `Monad`, later values can depend on previous ones. So, there is no `Monad` for `Validation`.
The `Applicative` apply for a type must not change behavior if derived from the `Monad` instance’s bind operation.
To derive `ap` from `Monad` instance:

In [9]:
ap :: (Monad m) => m (a -> b) -> m a -> m b
ap m m' = do
  x <- m
  x' <- m'
  return (x x')

The derivation shows the `m` and `m'` extractions are independent, hence `Applicative` does not require the full monadic strength (which use use `m` into `m'`)


### Short Exercise: Either Monad
Implement the `Either` `Monad`:

In [10]:
data Sum a b = First a | Second b deriving (Eq, Show)

instance Functor (Sum a) where
  fmap f (Second x) = Second $ f x

instance Applicative (Sum a) where
  pure = Second
  (Second f) <*> (Second x) = Second $ f x
  (First x)  <*> _          = First x
  _          <*> (First x)  = First x

instance Monad (Sum a) where
  return = pure
  (Second x) >>= f = f x
  (First x)  >>= _ = First x

## 18.5 Monad laws
### Identity laws
`Monad` has two identity laws:
1. right identity
`m >>= return = m`
2. left identity
`return x >>= f = f x`

### Associativity
`(m >>= f) >>= g=m >>= (\x -> f x >>= g)`


In [11]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
  fmap f (Identity x) = Identity $ f x

instance Applicative Identity where
  pure = Identity
  (Identity f) <*> (Identity x) = Identity $ f x

2. left identity:

`return x >>= f = f x`

### 18.6 Application and composition
In order to compose monadic functions it is not possible to use `.`. Instead, we need the _Kleisli composition_ which is:

`(>=>) :: Monad m => (a -> m b) -> (b -> m c) -> a -> m c`
### 18.7 Chapter exercises
Write `Monad` instances for the following types. Use the QuickCheck properties we showed you to validate your instances.
1. Welcome to the `Nope Monad`, where nothing happens and nobody cares:

In [12]:
data Nope a = NopeDotJpg deriving (Eq, Show)

instance Functor Nope where
  fmap _ _ = NopeDotJpg

instance Applicative Nope where
  pure _ = NopeDotJpg
  _ <*> _ = NopeDotJpg

instance Monad Nope where
  return = pure
  _ >>= _ = NopeDotJpg

instance Arbitrary a => Arbitrary (Nope a) where
  arbitrary = pure NopeDotJpg

instance Eq a => EqProp (Nope a) where
  (=-=) = eq

quickBatch $ monad NopeDotJpg


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.
  pure:           +++ OK, passed 500 tests.
  ap:             +++ OK, passed 500 tests.

2. 

In [13]:
data BahEither b a = PLeft a | PRight b deriving (Eq, Show)

instance Functor (BahEither b) where
  fmap _ (PRight x) = PRight x
  fmap f (PLeft x) = PLeft $ f x

instance Applicative (BahEither b) where
  pure = PLeft
  (PLeft f) <*> (PLeft x) = PLeft $ f x
  _ <*> (PRight x) = PRight x
  (PRight x) <*> _ = PRight x

instance Monad (BahEither b) where
  return = pure
  (PLeft x) >>= f = f x
  (PRight x) >>= _ = PRight x

instance (Arbitrary b, Arbitrary a) => Arbitrary (BahEither b a) where
  arbitrary = do
    f <- arbitrary
    x <- arbitrary
    elements [PLeft f, PRight x]
      
instance (Eq b, Eq a)  => EqProp (BahEither b a) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ monad (PLeft x)


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.
  pure:           +++ OK, passed 500 tests.
  ap:             +++ OK, passed 500 tests.

3. Write a `Monad` instance for `Identity`:

In [20]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
  fmap f (Identity x) = Identity $ f x

instance Applicative Identity where
  pure = Identity
  (Identity f) <*> (Identity x) = Identity $ f x

instance Monad Identity where
  return = pure
  (Identity x) >>= f = f x
  
instance (Arbitrary a) => Arbitrary (Identity a) where
  arbitrary = do
    x <- arbitrary
    pure $ Identity x
      
instance (Eq a) => EqProp (Identity a) where
  (=-=) = eq

x = ("1", "2", "3")
quickBatch $ monad (Identity x)


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.
  pure:           +++ OK, passed 500 tests.
  ap:             +++ OK, passed 500 tests.

4. 

In [21]:
data List a = Nil | Cons a (List a) deriving (Eq, Show)

instance Functor List where
  fmap _ Nil = Nil
  fmap f (Cons x xs) = Cons (f x) (fmap f xs)

instance Applicative List where
  pure x = Cons x Nil
  fs <*> xs = flatMap (<$> xs) fs

instance Monad List where
  return = pure
  (>>=) = flip flatMap
  
append :: List a -> List a -> List a
append Nil ys = ys
append (Cons x xs) ys = Cons x $ xs `append` ys

fold :: (a -> b -> b) -> b -> List a -> b
fold _ b Nil = b
fold f b (Cons h t) = f h (fold f b t)

concat' :: List (List a) -> List a
concat' = fold append Nil

flatMap :: (a -> List b) -> List a -> List b
flatMap f = concat' . fmap f 

instance (Arbitrary a) => Arbitrary (List a) where
  arbitrary = do
    x <- arbitrary
    pure $ Cons x Nil
      
instance Eq a => EqProp (List a) where
  xs =-= ys = xs' `eq` ys'
    where
      xs' = let l = xs in take' 100 l
      ys' = let l = ys in take' 100 l
      
take' :: Int -> List a -> List a
take' 0 _ = Nil
take' _ Nil = Nil
take' n (Cons x xs) = Cons x (take' (n - 1) xs)

x = ("1", "2", "3")
quickBatch $ monad (Cons x Nil)


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.
  pure:           +++ OK, passed 500 tests.
  ap:             +++ OK, passed 500 tests.

Write the following functions using the methods provided by `Monad` and `Functor`.

1. 

In [28]:
import Control.Monad (join)

j :: Monad m => m (m a) -> m a
j = (>>= id)

2. 

In [27]:
l1 :: Monad m => (a -> b) -> m a -> m b
l1 = fmap

3. 

In [26]:
import Control.Monad (liftM2)

l2 :: Monad m => (a -> b -> c) -> m a -> m b -> m c
l2 = liftM2

4. 

In [25]:
import Control.Monad (ap)

a :: Monad m => m a -> m (a -> b) -> m b
a = flip ap

5. You’ll need recursion for this one:

In [23]:
meh :: Monad m => [a] -> (a -> m b) -> m [b]
meh (x : xs) f = (:) <$> f x <*> meh xs f

6. Hint: reuse `meh`:

In [24]:
flipType :: Monad m => [m a] -> m [a]
flipType = flip meh id

## 18.8 Definitions
1. `Monad` is a type class reifying an abstraction consisting in functorially applying a function that produces more structure itself and using `join` to reduce the nested structure.
2. A _monadic function_ is one that generates more structure after having already been lifted over monadic structure.
3. _Bind_: could mean binding a value to a variable but in the context of monads it means `>>=`.